In [48]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')
import numpy as np

### Transportation Problem

#### North west corner method

In [282]:
def get_balanced(df):    
    supply=df['ai'].sum(skipna = True)
    demand=df.loc['bj'].sum(skipna = True)
    if supply==demand:
        return df
    elif supply<demand:
        #add dummy row            
        dummy_row = pd.DataFrame({key:0 for key in df.columns}, index=["O"+str(n+1)])
        dummy_row['ai'][0]=demand-supply
        return pd.concat([df.iloc[:-1], dummy_row, df.iloc[-1:]])       
        
    else:#supply>demand
        #add dummy column
        dummy_col =pd.DataFrame({"D"+str(m+1):[0 for val in range(m)]},index=df.index)
        dummy_col["D"+str(m+1)][-1]=supply-demand
        return pd.concat([df.iloc[:,:-1], dummy_col, df.iloc[:,-1:]],axis=1)
        


In [283]:
def modify_table(df,cost,cost_table):    
    #print('###################')
    #print(df)
    #print('###################')
    j=len(df.columns)-1
    i=len(df.index)-1
    if df['ai'][0]>df[df.columns[0]]['bj']:
        df['ai'][0]-=df[df.columns[0]]['bj']   
        cost+=(df.iloc[0,0]*df[df.columns[0]]['bj'])
        #print("x",x,"i",i,"y",y,"j",j)
        #print("col",df.columns[0])
        cost_table[n-i][m-j]=df[df.columns[0]]['bj']        
        return df.drop([df.columns[0]],axis=1),cost,cost_table
    elif df['ai'][0]<df[df.columns[0]]['bj']:
        df[df.columns[0]]['bj']-=df['ai'][0]   
        cost+=(df.iloc[0,0]*df['ai'][0])
        #print("x",x,"i",i,"y",y,"j",j)
        cost_table[n-i][m-j]=df['ai'][0]        
        return df.drop(df.index[0]),cost,cost_table
    else:
        cost+=(df.iloc[0,0]*df['ai'][0])
        cost_table[n-i][m-j]=df['ai'][0]
        df=df.drop(df.index[0])
        #print("x",x,"i",i,"y",y,"j",j)        
        return df.drop([df.columns[0]],axis=1),cost,cost_table
        
        


In [284]:
def print_cost_table(cost_table):
    for row in cost_table:
        for val in row:
            print(str(val).rjust(10),end=" ")
        print()

In [285]:
def north_west_corner_method(df):    
    cost_table=[[0 for i in range(m)] for j in range(n)]
    cost=0
    allocated_cells=0
    print(df)
    print("____________________________________________________________")
    for i in range(10):
        if df.columns[0]=='ai' or df.index[0]=='bj':
            print("Terminating.....")
            print("Cost after NWCmethod:", cost)
            break
        else:
            df,cost,cost_table=modify_table(df,cost,cost_table)
            allocated_cells+=1
            print("The new transportation table is:")
            print(df)
            print("------------------------")
            print("Cost table:")
            print_cost_table(cost_table)
            print("____________________________________________________________")
    #print("cost table*************",cost_table)
    return cost, cost_table,allocated_cells
            

### getting non-degenerate table

In [286]:
#e->epsilon value to be allocated in case the cell is degenarate
e=0.0000001
infinity=999999999

In [287]:
def FindPath( u, v, cost_table,df):    
    #i,j=4,4
    aPath = [[ u, v]]
    path_found=LookHorizontaly( aPath, u, v, u, v,cost_table)       
    return path_found,aPath

def LookHorizontaly( aPath, u, v, u1, v1,cost_table):
    for i in range( 0, m):
        #print('u',u,'i',i)
        if i != v and cost_table[ u][ i] != 0:
            if i == v1:
                aPath.append( [ u, i])
                return True # complete circuit
            if LookVerticaly( aPath, u, i, u1, v1,cost_table):
                aPath.append( [ u, i])
                return True
    return False # not found

def LookVerticaly( aPath, u, v, u1, v1,cost_table):
    for i in range( 0, n):
        #print('v',v,'i',i)
        if i != u and cost_table[ i][ v] != 0:
            if LookHorizontaly( aPath, i, v, u1, v1,cost_table):
                aPath.append([ i, v])
                return True
    return False # not found

In [288]:
def next_smallest(prev_smallest,df,prev_i,prev_j,cost_table):    
    val_min=infinity
    min_i=-1
    min_j=-1
    for i in range(n):
        for j in range(m):
            if (prev_i!=i or prev_j!=j) and df.iloc[i,j]>=prev_smallest and cost_table[i][j]==0:
                if val_min>df.iloc[i,j]:
                    val_min=df.iloc[i,j]
                    min_i=i
                    min_j=j
    if val_min==infinity:
        raise Exception("next smallest value is infinity")
    return val_min,min_i,min_j
                    
                

def check_degeneracy(allocated_cells,df,cost_table): 
    print("Checking for degeneracy")
    prev_smallest=-infinity
    prev_i=-1
    prev_j=-1
    for num in range(10):
        if (m+n-1)-allocated_cells==0:#non-degenerate
            print("Problem is now non-degenerate")
            return cost_table
        elif (m+n-1)-allocated_cells>0:#degenerate
            smallest,u,v=next_smallest(prev_smallest,df,prev_i,prev_j,cost_table)
            prev_smallest=smallest
            prev_i=u
            prev_j=v
            path_found,aPath=FindPath( u, v, cost_table,df)
            if not path_found:                
                cost_table[u][v]=e
                print('cost table after allocating cell:',u,v)
                print_cost_table(cost_table)
                allocated_cells+=1
        else:
            print("Some error occured in degeneracy1")
            return cost_table
    print("Some error occured in degeneracy2")
    return cost_table
        
        
        
    

### Stepping stone method

In [289]:
def path_cost(aPath,df):
    pcost=0
    one=1
    for val in aPath:
        pcost+=(one*df.iloc[val[0],val[1]])
        one*=(-1)
    return pcost
        
    
def check_optimality(df,cost_table):    
    min_pcost=infinity    
    min_path=None
    for i in range(n):
        for j in range(m):
            if cost_table[i][j]==0:
                path_found,aPath=FindPath( i, j, cost_table,df)
                if path_found:                    
                    pcost=path_cost(aPath,df)                    
                    print("Path",aPath)
                    print("path cost", pcost)
                    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                    if min_pcost>pcost:
                        min_pcost=pcost                        
                        min_path=aPath
    if min_pcost==infinity:
        raise Exception("min_pcost is infinity")
    return min_pcost,min_path

def modify_cost_table(cost_table,min_path):
    n0=len(min_path)
    print("Minimum Path:",min_path)  
    neg_val=[cost_table[min_path[i][0]][min_path[i][1]] for i in range(1,n0,2)]
    min_val=min(neg_val)
    degeneracy_count=neg_val.count(min_val)
    for i in range(0,n0,2):
        cost_table[min_path[i][0]][min_path[i][1]]+=min_val
    for i in range(1,n0,2):
        cost_table[min_path[i][0]][min_path[i][1]]-=min_val
    if degeneracy_count>1:
        cost_table=check_degeneracy(m+n-degeneracy_count,df,cost_table)
    return cost_table


def transport_cost(df,cost_table):   
    cost=0
    for i in range(n):
        for j in range(m):
            cost+=(df.iloc[i,j]*cost_table[i][j])
    return cost

def stepping_stone_method(df,cost_table):
    for i in range(10):
        min_pcost,min_path=check_optimality(df,cost_table)
        if min_pcost>=0:
            print("-----------------------------------------")
            print("Cost table:")
            print_cost_table(cost_table)
            print("-----------------------------------------")
            print("The minimum transportation cost is", transport_cost(df,cost_table))       
            break
        else:
            cost_table=modify_cost_table(cost_table,min_path)
            print("-----------------------------------------")
            print("Cost table:")
            print_cost_table(cost_table)
            print("-----------------------------------------")
    
    
                    
                
                
    
    

In [290]:
def transportation_problem(df):
    #df=get_balanced(unbalanced_df)
    df_temp=df.copy(deep=True)
    cost,cost_table,allocated_cells=north_west_corner_method(df_temp)
    print("Proceeding to stepping stone method:......")
    cost_table=check_degeneracy(allocated_cells,df,cost_table)
    stepping_stone_method(df,cost_table)

In [292]:
#1
df=pd.read_csv("prob1.csv") 
df = df.set_index('id').astype(float)
df=get_balanced(df)
m=len(df.columns)-1
n=len(df.index)-1
transportation_problem(df)



      D1   D2   D3   D4   ai
id                          
O1  10.0  7.0  3.0  6.0  3.0
O2   1.0  6.0  8.0  3.0  5.0
O3   7.0  4.0  5.0  3.0  7.0
bj   3.0  2.0  6.0  4.0  NaN
____________________________________________________________
The new transportation table is:
     D2   D3   D4   ai
id                    
O2  6.0  8.0  3.0  5.0
O3  4.0  5.0  3.0  7.0
bj  2.0  6.0  4.0  NaN
------------------------
Cost table:
       3.0          0          0          0 
         0          0          0          0 
         0          0          0          0 
____________________________________________________________
The new transportation table is:
     D3   D4   ai
id               
O2  8.0  3.0  3.0
O3  5.0  3.0  7.0
bj  6.0  4.0  NaN
------------------------
Cost table:
       3.0          0          0          0 
         0        2.0          0          0 
         0          0          0          0 
____________________________________________________________
The new transportation table

In [293]:
#2
df=pd.read_csv("prob2.csv") 
df = df.set_index('id').astype(float)
df=get_balanced(df)
m=len(df.columns)-1
n=len(df.index)-1
transportation_problem(df)

      D1    D2    D3    D4    ai
id                              
O1  19.0  30.0  50.0  10.0   7.0
O2  70.0  30.0  40.0  60.0   9.0
O3  40.0   8.0  70.0  20.0  18.0
bj   5.0   8.0   7.0  14.0   NaN
____________________________________________________________
The new transportation table is:
      D2    D3    D4    ai
id                        
O1  30.0  50.0  10.0   2.0
O2  30.0  40.0  60.0   9.0
O3   8.0  70.0  20.0  18.0
bj   8.0   7.0  14.0   NaN
------------------------
Cost table:
       5.0          0          0          0 
         0          0          0          0 
         0          0          0          0 
____________________________________________________________
The new transportation table is:
      D2    D3    D4    ai
id                        
O2  30.0  40.0  60.0   9.0
O3   8.0  70.0  20.0  18.0
bj   6.0   7.0  14.0   NaN
------------------------
Cost table:
       5.0        2.0          0          0 
         0          0          0          0 
         0        

In [294]:
#3
df=pd.read_csv("prob3.csv") 
df = df.set_index('id').astype(float)
df=get_balanced(df)
m=len(df.columns)-1
n=len(df.index)-1
transportation_problem(df)

      D1    D2    D3    D4    ai
O1   3.0   8.0   7.0   4.0  30.0
O2   5.0   2.0   9.0   5.0  50.0
O3   4.0   3.0   6.0   2.0  80.0
O4   0.0   0.0   0.0   0.0  15.0
bj  20.0  60.0  55.0  40.0   NaN
____________________________________________________________
The new transportation table is:
      D2    D3    D4    ai
O1   8.0   7.0   4.0  10.0
O2   2.0   9.0   5.0  50.0
O3   3.0   6.0   2.0  80.0
O4   0.0   0.0   0.0  15.0
bj  60.0  55.0  40.0   NaN
------------------------
Cost table:
      20.0          0          0          0 
         0          0          0          0 
         0          0          0          0 
         0          0          0          0 
____________________________________________________________
The new transportation table is:
      D2    D3    D4    ai
O2   2.0   9.0   5.0  50.0
O3   3.0   6.0   2.0  80.0
O4   0.0   0.0   0.0  15.0
bj  50.0  55.0  40.0   NaN
------------------------
Cost table:
      20.0       10.0          0          0 
         0        